In [2]:
#Importing Reuired Libraries

import numpy as np
import pandas as pd
from pyomo.environ import *
import matplotlib.pyplot as plt
import time
import networkx as nx

Required paths

In [24]:
glpk_solver_exe_file = 'C:\\Users\\Ashish\\winglpk-4.65\\glpk-4.65\\w64\\glpsol.exe'

name_of_input_file = 'tsp29.txt'

Writing required functions

In [3]:
#function to reverse a tuple
def Reverse(tuples):
    new_tup = tuples[::-1]
    return new_tup

#function to detect cycles from solution and returning nested list of cycles with
#first list containg (0,j)&(i,0)
def cycle_det(active_arcs):
    duplicat_arcs=list()
    for i,j in active_arcs:
        duplicat_arcs.append((i,j))
    cycle_iter_var = 0
    cycle_list = list()
    while True:
        g = nx.Graph()
        g.add_edges_from(duplicat_arcs)
        try:
            cycle = nx.find_cycle(g)
            #print(f'Cycle {cycle_iter_var} is ______{cycle}')
            cycle_list.append(cycle)
        except:
            #print('all arcs done')
            break
        for i,j in cycle:
            duplicat_arcs.remove((i,j))
    cycle_iter_var = cycle_iter_var+1
    cycle_list_start_with_0=[]
    for i in cycle_list:
        for tup in i:
            if tup[0]==0 or tup[1]==0:
                cycle_list_start_with_0.append(i)
                continue
        cycle_list_start_with_0.append(i)
    if len(cycle_list)==1:
        return 0
    else:
        return cycle_list_start_with_0

Reading the input files

In [4]:
#taking user input
fh = open(f'Input\\{name_of_input_file}')
#marking start time
start_time = time.time()
#reading the text file
loc_x = list()
loc_y = list()
for i in fh:
    a = i.split()
    loc_x.append(float(a[1]))
    loc_y.append(float(a[2]))
n = len(loc_x)
#creating nodes and edges
cities = [i for i in range(n)]
edges = [(i,j) for i in cities for j in cities if i!=j]
#input plot
plt.figure(figsize=(20, 10))
plt.scatter(loc_x,loc_y,color='blue')
plt.xlabel("Distance X")
plt.ylabel("Distance Y")
plt.title("Data points for Djibouti")
s = []
for n in range(len(loc_x)):
    s_temp= []
    s_temp.append("%.lf"%loc_x[n])
    s_temp.append("%.lf"%loc_y[n])
    s.append(s_temp)
#for n in range(len(loc_x)):
#   plt.annotate(str(s[n]),xy=(loc_x[n],loc_y[n]),xytext=(loc_x[n]-4,loc_y[n]-5), color='red')
for n in range(len(loc_x)):
    plt.annotate(str(n),xy=(loc_x[n],loc_y[n]),xytext=(loc_x[n]+4,loc_y[n]+10), color='red')
plt.savefig('Output\\Images\\frame_0.png')

#Calculate euclidiean distance
distance  = {(i,j):np.hypot(loc_x[i]-loc_x[j],loc_y[i]-loc_y[j]) for i,j in edges}

Creating the MILP Model

In [5]:
# Defining the model
model = ConcreteModel()

# Defining sets
model.cities = Set(initialize=cities)
model.edges = Set(initialize=edges, dimen=2)

Defining Variables and Objective function

In [6]:
# Defining variable for model
model.x = Var(model.edges, within=Binary)

# Objective function
def obj_rule(model):
    return sum(distance[i, j] * model.x[i, j] for i, j in model.edges)

model.obj = Objective(rule=obj_rule, sense=minimize)

Writing required constraints

In [7]:
# Constraints
def out_degree_rule(model, c):
    return sum(model.x[i, j] for i, j in model.edges if i == c) == 1

def in_degree_rule(model, c):
    return sum(model.x[i, j] for i, j in model.edges if j == c) == 1

model.out_degree = Constraint(model.cities, rule=out_degree_rule)
model.in_degree = Constraint(model.cities, rule=in_degree_rule)

# Subtour elimination for 2 cities
def subtour_elimination_rule(model, i, j, k, l):
    if i == l and j == k:
        return model.x[i, j] + model.x[k, l] <= 1
    return Constraint.Skip

model.subtour_elimination = Constraint(model.edges, model.edges, rule=subtour_elimination_rule)

Iniial solve of TSP without Subtour Elemination Constraints

In [8]:
# Solving the problem initially
solver = SolverFactory('glpk',executable=glpk_solver_exe_file)
solution = solver.solve(model, tee=True)


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\Ashish\AppData\Local\Temp\tmp4vdix33w.glpk.raw --wglp C:\Users\Ashish\AppData\Local\Temp\tmpz3ru207z.glpk.glp
 --cpxlp C:\Users\Ashish\AppData\Local\Temp\tmpydx_xxtd.pyomo.lp
Reading problem data from 'C:\Users\Ashish\AppData\Local\Temp\tmpydx_xxtd.pyomo.lp'...
C:\Users\Ashish\AppData\Local\Temp\tmpydx_xxtd.pyomo.lp:7492: warning: lower bound of variable 'x2' redefined
C:\Users\Ashish\AppData\Local\Temp\tmpydx_xxtd.pyomo.lp:7492: warning: upper bound of variable 'x2' redefined
870 rows, 812 columns, 3248 non-zeros
812 integer variables, all of which are binary
8304 lines were read
Writing problem data to 'C:\Users\Ashish\AppData\Local\Temp\tmpz3ru207z.glpk.glp'...
6615 lines were written
GLPK Integer Optimizer, v4.65
870 rows, 812 columns, 3248 non-zeros
812 integer variables, all of which are binary
Preprocessing...
870 rows, 812 columns, 3248 non-zeros
812 integer variables, all of which a

Solving TSP iteratively, ST after each loop we detect the subtours created and solve the same

In [10]:
# Extracting the active arcs
active_arcs = [(i, j) for i, j in model.edges if model.x[i, j].value > 0.9]

model.subtour_elimination_dynamic = ConstraintList()
# Solving in loop that is adding necessary SEC
frame = 1
while True:
    x_1 = cycle_det(active_arcs)
    if x_1 == 0:
        print('Optimal path found')
        optimal_path = active_arcs
        print('Objective value is', value(model.obj))
        print(f'Code took {(time.time() - start_time)} seconds to execute')
        print(f'Active arcs are {optimal_path}')
        break
    else:
        for i in range(len(x_1) - 1):
            i = (len(x_1) - 1) - i
            sub_cycle = x_1[i]
            model.subtour_elimination_dynamic.add(sum(model.x[j] for j in sub_cycle) <= len(sub_cycle) - 1)
            model.subtour_elimination_dynamic.add(sum(model.x[Reverse(j)] for j in sub_cycle) <= len(sub_cycle) - 1)
            #Plot intermediate plots
            plt.figure(figsize=(20, 10))
            plt.xlabel("Distance X")
            plt.xlabel("Distance Y")
            plt.title("Intermediate Subtours")
            plt.scatter(loc_x, loc_y, color='blue', zorder=1)
            for i, j in active_arcs:
                plt.plot([loc_x[i], loc_x[j]], [loc_y[i], loc_y[j]], color='blue', zorder=1)
            for n in range(len(loc_x)):
                plt.annotate(str(n), xy=(loc_x[n], loc_y[n]), xytext=(loc_x[n] + 4, loc_y[n] + 10), color='red')
            plt.savefig(f'Output\\Images\\frame_{frame}.png')
            frame += 1
        solution = solver.solve(model, tee=True)
        active_arcs = [(i, j) for i, j in model.edges if model.x[i, j].value > 0.9]

# Plotting the output
plt.figure(figsize=(20, 10))
plt.xlabel("Distance X")
plt.xlabel("Distance Y")
plt.title("Solution to the TSP")
plt.scatter(loc_x, loc_y, color='blue', zorder=1)
for i, j in active_arcs:
    plt.plot([loc_x[i], loc_x[j]], [loc_y[i], loc_y[j]], color='blue', zorder=1)
for n in range(len(loc_x)):
    plt.annotate(str(n), xy=(loc_x[n], loc_y[n]), xytext=(loc_x[n] + 4, loc_y[n] + 10), color='red')
plt.savefig(f'Output\\Images\\frame_{frame}.png')

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\Ashish\AppData\Local\Temp\tmpantya37n.glpk.raw --wglp C:\Users\Ashish\AppData\Local\Temp\tmpcqp2annl.glpk.glp
 --cpxlp C:\Users\Ashish\AppData\Local\Temp\tmpx_zu7l73.pyomo.lp
Reading problem data from 'C:\Users\Ashish\AppData\Local\Temp\tmpx_zu7l73.pyomo.lp'...
C:\Users\Ashish\AppData\Local\Temp\tmpx_zu7l73.pyomo.lp:7610: warning: lower bound of variable 'x2' redefined
C:\Users\Ashish\AppData\Local\Temp\tmpx_zu7l73.pyomo.lp:7610: warning: upper bound of variable 'x2' redefined
888 rows, 812 columns, 3312 non-zeros
812 integer variables, all of which are binary
8422 lines were read
Writing problem data to 'C:\Users\Ashish\AppData\Local\Temp\tmpcqp2annl.glpk.glp'...
6715 lines were written
GLPK Integer Optimizer, v4.65
888 rows, 812 columns, 3312 non-zeros
812 integer variables, all of which are binary
Preprocessing...
18 hidden covering inequaliti(es) were detected
888 rows, 812 columns, 3312

c:\Users\Ashish\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\Ashish\AppData\Local\Temp\tmp01j0l7t5.glpk.raw --wglp C:\Users\Ashish\AppData\Local\Temp\tmpct9i7yqf.glpk.glp
 --cpxlp C:\Users\Ashish\AppData\Local\Temp\tmprw1fgo37.pyomo.lp
Reading problem data from 'C:\Users\Ashish\AppData\Local\Temp\tmprw1fgo37.pyomo.lp'...
C:\Users\Ashish\AppData\Local\Temp\tmprw1fgo37.pyomo.lp:7940: warning: lower bound of variable 'x2' redefined
C:\Users\Ashish\AppData\Local\Temp\tmprw1fgo37.pyomo.lp:7940: warning: upper bound of variable 'x2' redefined
914 rows, 812 columns, 3564 non-zeros
812 integer variables, all of which are binary
8752 lines were read
Writing problem data to 'C:\Users\Ashish\AppData\Local\Temp\tmpct9i7yqf.glpk.glp'...
7019 lines were written
GLPK Integer Optimizer, v4.65
914 rows, 812 columns, 3564 non-zeros
812 integer variables, all of which are binary
Preprocessing...
44 hidden covering inequaliti(es) were detected
914 rows, 812 columns, 3564

Writing the result file

Creating the video from the Images

In [11]:
#Creating a Video of the Images

import os
from PIL import Image
import cv2

current_path = os.getcwd()
print(current_path)
img_dir = "Output\\Images"

def compute_average_dimensions(folder):
    total_width = 0
    total_height = 0
    img_count = 0

    for img_file in os.listdir(folder):
        if img_file.endswith((".jpg", ".jpeg", ".png")):
            image = Image.open(os.path.join(folder, img_file))
            w, h = image.size
            total_width += w
            total_height += h
            img_count += 1

    avg_width = int(total_width / img_count)
    avg_height = int(total_height / img_count)
    return avg_width, avg_height

# Calculate average dimensions of images
avg_width, avg_height = compute_average_dimensions(img_dir)


video_filename = 'Output\\created_video.mp4'
valid_images = [i for i in os.listdir(img_dir) if i.endswith((".jpg", ".jpeg", ".png"))]

valid_images = [f'frame_{i}.png' for i in range(0,len(valid_images))]

first_image = cv2.imread(os.path.join(img_dir, valid_images[0]))
h, w, _ = first_image.shape

codec = cv2.VideoWriter_fourcc(*'mp4v')
vid_writer = cv2.VideoWriter(video_filename, codec, 60, (w, h))

for img in valid_images:
    loaded_img = cv2.imread(os.path.join(img_dir, img))
    for _ in range(20):
        vid_writer.write(loaded_img)

vid_writer.release()

e:\Self Projects\MILP\Travelling Salesman Problem
